<a href="https://colab.research.google.com/github/Vattikolla/FMML-2024/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
If you increase the percentage of the validation set:

1. _More robust evaluation_: A larger validation set provides a more accurate estimate of your model's performance.
2. _Better hyperparameter tuning_: With more data, you can fine-tune hyperparameters more effectively.
3. _Overfitting detection_: A larger validation set helps detect overfitting by showing a larger gap between training and validation performance.
4. _Increased computational cost_: Training and evaluating on a larger validation set takes more time and resources.

If you reduce the percentage of the validation set:

1. _Less robust evaluation_: A smaller validation set may not accurately represent your model's performance.
2. _Poorer hyperparameter tuning_: With less data, hyperparameter tuning may be less effective.
3. _Overfitting risk_: A smaller validation set may mask overfitting, making it harder to detect.
4. _Faster training and evaluation_: Training and evaluating on a smaller validation set takes less time and resources.

In general, a validation set size of 15% to 30% of the total dataset is a good starting point. However, this can vary depending on the specific problem, dataset size, and computational resources.

Remember, the key is to strike a balance between having enough data for robust evaluation and not using too much data, which could lead to overfitting or increased computational costs.
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
A.The size of the train and validation sets significantly impacts how well you can predict the accuracy on the test set using the validation set. Here's how:

*Train Set Size:*

1. *Small train set*: With a small train set, the model may not learn enough to generalize well to new data, leading to poor performance on both validation and test sets.
2. *Large train set*: A large train set provides more data for the model to learn from, improving its ability to generalize and predict accuracy on the test set.

*Validation Set Size:*

1. *Small validation set*: A small validation set may not accurately represent the test set, leading to overestimation or underestimation of performance.
2. *Large validation set*: A larger validation set provides a more accurate estimate of performance, allowing for better hyperparameter tuning and model selection.

*Ideal Split:*

1. *Train set*: 70-80% of the total dataset
2. *Validation set*: 15-25% of the total dataset
3. *Test set*: 5-15% of the total dataset

With this split, you can:

1. Train a robust model on a large train set
2. Accurately estimate performance on a sizable validation set
3. Evaluate final performance on a separate test set

Remember, the key is to strike a balance between train, validation, and test set sizes to ensure reliable performance estimation and model generalization.
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?
A.A good percentage to reserve for the validation set depends on the dataset size and complexity. However, here are some general guidelines:

- For small datasets (<10,000 samples), reserve 20-30% for validation
- For medium datasets (10,000-100,000 samples), reserve 15-25% for validation
- For large datasets (>100,000 samples), reserve 10-20% for validation

Reserving too little for validation (e.g., 5%) may lead to overestimation of performance, while reserving too much (e.g., 50%) may reduce the train set size, hindering model learning.

Consider the following factors when choosing a validation set percentage:

- Dataset size and complexity
- Model complexity and capacity
- Computational resources and training time
- Desired level of performance estimation accuracy

Remember, these are general guidelines. You may need to experiment with different validation set sizes to find the sweet spot for your specific problem.

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
A.Yes, averaging the validation accuracy across multiple splits can provide more consistent results. This technique is called "cross-validation". By training and evaluating the model on multiple splits, you can:

1. Reduce variability in performance estimates
2. Increase confidence in the model's ability to generalize
3. Get a more accurate estimate of the model's performance on unseen data

Common cross-validation techniques include:

1. K-Fold Cross-Validation: Split data into k folds, train on k-1 folds, and evaluate on the remaining fold. Repeat for each fold.
2. Stratified Cross-Validation: Similar to k-fold, but maintains class balance in each fold.
3. Monte Carlo Cross-Validation: Randomly splits data into train and validation sets, repeats multiple times.

Averaging validation accuracy across multiple splits helps to:

1. Reduce overfitting to a single split
2. Increase robustness to outliers or noisy data
3. Provide a more reliable estimate of performance

Remember, cross-validation is computationally expensive, so balance the number of splits with computational resources and training time.
2. Does it give more accurate estimate of test accuracy?
A.Yes, averaging validation accuracy across multiple splits can provide a more accurate estimate of test accuracy. By reducing variability in performance estimates, cross-validation helps to:

1. Minimize overestimation or underestimation of performance
2. Account for variability in data and model training
3. Provide a more robust estimate of how well the model will perform on unseen data (test set)

Cross-validation can help to:

1. Reduce the gap between validation and test accuracy
2. Increase confidence in the model's ability to generalize to new data
3. Provide a more reliable estimate of test accuracy, which is essential for model deployment and decision-making

However, keep in mind that:

1. Cross-validation is not a guarantee of perfect accuracy estimation
2. Other factors like data quality, model complexity, and hyperparameter tuning still impact performance
3. Test accuracy can still be affected by factors like data drift, concept drift, or unseen scenarios

By using cross-validation, you can increase the reliability of your performance estimates and make more informed decisions about your model's deployment and use.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
A.The number of iterations in cross-validation can impact the estimate of test accuracy:

1. *Increased iterations*: Generally, more iterations lead to a more accurate estimate of test accuracy.
2. *Reduced variability*: More iterations reduce the variability in performance estimates, providing a more stable and reliable estimate.
3. *Convergence*: With more iterations, the estimate converges to the true test accuracy, assuming the model and data are stable.

However, there are diminishing returns:

1. *Law of diminishing returns*: Beyond a certain point (e.g., 10-20 iterations), additional iterations may not significantly improve the estimate.
2. *Computational cost*: More iterations increase computational time and resources.

A good starting point is to use:

1. 5-10 iterations for small datasets or simple models
2. 10-20 iterations for medium datasets or moderate models
3. 20-50 iterations for large datasets or complex models

Remember, the optimal number of iterations depends on the specific problem, dataset size, and model complexity. You may need to experiment to find the sweet spot.

Higher iterations can lead to a better estimate, but it's essential to balance iterations with computational resources and time constraints.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
A .Increasing iterations can help to some extent when dealing with a very small train or validation dataset, but it has limitations:

1. _Small train dataset_: Increasing iterations can help reduce overfitting, but it may not fully compensate for the lack of training data. More data is still necessary for robust model learning.
2. _Small validation dataset_: Increasing iterations can help reduce variability in performance estimates, but it may not provide a reliable estimate of test accuracy if the validation set is too small.

In such cases, consider:

1. _Data augmentation_: Artificially increase the size of the train dataset by applying transformations (e.g., rotation, scaling) to existing samples.
2. _Transfer learning_: Leverage pre-trained models and fine-tune them on your small dataset.
3. _Active learning_: Selectively sample new data points to add to the train or validation sets.
4. _Collect more data_: If possible, collect additional data to increase the size of the train and validation sets.
5. _Model simplification_: Use simpler models or reduce model complexity to adapt to the small dataset size.

Remember, increasing iterations is not a substitute for a sufficient dataset size. If the dataset is too small, consider the above strategies to improve model performance and reliability.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.